In [1]:
#Alphavantage API Testing
import requests
from bs4 import BeautifulSoup
import pandas as pd

# --- SETTINGS ---
symbol = "AAPL"  # Example: Apple
ALPHA_KEY = "YOUR_ALPHA_VANTAGE_API_KEY"

# --- TRY ALPHA VANTAGE FIRST ---
try:
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={ALPHA_KEY}"
    r = requests.get(url, timeout=20)
    r.raise_for_status()
    js = r.json()

    key = [k for k in js.keys() if "Time Series" in k]
    assert key, f"Unexpected response keys: {list(js.keys())}"

    series = js[key[0]]
    df_api = (
        pd.DataFrame(series).T
        .rename_axis("date")
        .reset_index()
        .rename(columns={
            "1. open": "Open",
            "2. high": "High",
            "3. low": "Low",
            "4. close": "Close",
            "5. volume": "Volume"
        })
    )

    print("✅ Data from Alpha Vantage API")
    print(df_api.head())

except Exception as e:
    print("⚠ API failed, falling back to Yahoo Finance scrape:", e)

    # --- FALLBACK: SCRAPE YAHOO FINANCE ---
    scrape_url = f"https://finance.yahoo.com/quote/{symbol}/history"
    headers = {"User-Agent": "Mozilla/5.0"}
    
    resp = requests.get(scrape_url, headers=headers, timeout=20)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")
    table = soup.find("table")
    
    rows = []
    for tr in table.find_all("tr"):
        cells = [td.get_text(strip=True) for td in tr.find_all(["td", "th"])]
        if cells:
            rows.append(cells)
    
    header, *data = rows
    df_scrape = pd.DataFrame(data, columns=header)

    print("✅ Data from Yahoo Finance scraping")
    print(df_scrape.head())


✅ Data from Alpha Vantage API
         date      Open      High       Low     Close    Volume
0  2025-08-22  226.1700  229.0900  225.4100  227.7600  42477811
1  2025-08-21  226.2700  226.5200  223.7804  224.9000  30621249
2  2025-08-20  229.9800  230.4700  225.7700  226.0100  42263865
3  2025-08-19  231.2750  232.8700  229.3500  230.5600  39402564
4  2025-08-18  231.7000  233.1200  230.1100  230.8900  37476188
